<a href="https://colab.research.google.com/github/aknip/Coding-Cheatsheets/blob/main/Python-AI-LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai tiktoken litellm langchain icecream --quiet

In [ ]:
import json
import os
from getpass import getpass
import psutil
import requests
import textwrap
from icecream import ic
import time
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [ ]:
import openai
from litellm import completion
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v1']['credential'] # my key
os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']

# Simple LLM calls

In [ ]:
import openai
import os

client = openai.OpenAI(
  api_key=os.environ.get("TOGETHERAI_API_KEY"),
  base_url='https://api.together.xyz',
)

chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "user",
      "content": "Hello, how are you?",
    }
  ],
  model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  max_tokens=1024
)

print(chat_completion.choices[0].message.content)

In [ ]:
# Call via lite-llm

response = completion(
  #model="together_ai/togethercomputer/Llama-2-7B-32K-Instruct",
  model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

In [ ]:
response = completion(
  model="gpt-3.5-turbo",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(response.choices[0].message.content)
print(response)

In [ ]:
# Function Calling

# see https://litellm.vercel.app/docs/completion/function_call

# via Huggingface?
# https://litellm.vercel.app/docs/providers/huggingface
# https://huggingface.co/Trelis/Mixtral-8x7B-Instruct-v0.1-function-calling-v3
# https://huggingface.co/Trelis/Mistral-7B-Instruct-v0.1-function-calling-v2

# via Anyscale?
# https://docs.litellm.ai/docs/providers/anyscale
# https://www.anyscale.com/blog/anyscale-endpoints-json-mode-and-function-calling-features

import os, litellm
from litellm import completion

# IMPORTANT - Set this to TRUE to add the function to the prompt for Non OpenAI LLMs
litellm.add_function_to_prompt = True

# The real function is not needed for the LLM. It may be called after the LLM call (not in this code!)
def get_current_weather(location):
  if location == "Boston, MA":
    return "The weather is 12F"

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]

messages = [
    {"role": "user", "content": "What is the weather like in Boston?"}
]

llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-3.5-turbo-1106"
#llm_model = "gpt-4"
#llm_model = "claude-2" # ??? claude answers in response.choices[0]['message']['content']
#llm_model = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"


response = completion(model=llm_model, messages=messages, functions=functions)

function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_arguments)
else:
  if llm_model == "claude-2":
    print(response.choices[0]['message']['content'])
  else:
    print('No function found')

print('\n')
print(response.choices[0]['message']['content'])
print('\n')
print(json.dumps(response.json(), indent=2))

In [ ]:
import os, litellm
from litellm import completion

litellm.add_function_to_prompt = False

functions_string = """[
  {
    "name": "get_risk_data",
    "description": "Daten zum Versicherungsnehmer und seiner Risiken",
    "parameters": {
      "type": "object",
      "properties": {
        "Versicherungssumme": {
          "type": "integer",
          "description": "Versicherungssumme oder Deckungssumme der Versicherung, z. B. 100000"
        },
        "Versicherungsnehmer_Name": {
          "type": "string",
          "description": "Name des versicherten Unternehmens, z. B. 'Testfirma GmbH'"
        },
        "Versicherungsnehmer_Anschrift": {
          "type": "string",
          "description": "Anschrift des versicherten Unternehmens: Strasse, PLZ, Ort, z. B. 'Musterstr. 10, 10000 Berlin'"
        },
        "Versicherungsnehmer_Branche": {
          "type": "string",
          "description": "Branche bzw. Tätigkeitsbereich des versicherten Unternehmens, z. B. 'Textilproduktion'"
        },
        "Versicherungsnehmer_Umsatz": {
          "type": "string",
          "description": "Jahresumsatz des versicherten Unternehmens, z. B. 23000000"
        },
        "Vorschäden_benannt": {
          "type": "string",
          "description": "Informationen zu früheren Schadensfällen, z. B. 'Ja, in 2019 gab es einen Schaden mit folgenden Details'"
        },
      },
      "required": ["Versicherungssumme"]
    }
  }
]
"""

functions=eval(functions_string)

email_test_text = """wir hoffen, Sie hatten einen angenehmen Feiertag.

Zu unserer besteMeiermen Kundenverbindung Zorrsen Beteiligungsgesellschaft mbH bitten wir um Ihre Quotierung zur D&O-Versicherung per 01.01.2024.

Die aktuelle Versicherungssumme beträgt 5,5 Mio. € zu einer Jahresprämie in Höhe von 7.920 € netto.
Der Umsatz im letzten Jahr betrug 78,5 Millionen EUR.

Beigefügt erhalten Sie die bisherigen Vertragsunterlagen sowie ein Organigramm. Versichert werden soll die Zorrsen Beteiligungsgesellschaft mbH als VN und die darunterliegenden Tochtergesellschaften (gelb markiert).

Es gibt einen laufenden D&O-Schadensfall aufgrund einer Inanspruchnahme eines ehemaligen Geschäftsführers, welcher im Jahr 2017 entlassen wurde. Ihm wird vorgeworfen, für ein damaliges Bauvorhaben das Budget deutlich überzogen zu haben.

Für Ihre Rückmeldung vielen Dank im Voraus.
"""

messages = [
    {"role": "user", "content": "Extrahiere die Risikodaten: " + email_test_text}
]

llm_model = "gpt-3.5-turbo"
#llm_model = "gpt-3.5-turbo-1106"
#llm_model = "gpt-4"
#llm_model = "claude-2" # ??? claude answers in response.choices[0]['message']['content']
#llm_model = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"


response = completion(model=llm_model, messages=messages, functions=functions)

function_found = hasattr(response.choices[0]['message'], 'function_call')
if function_found == True:
  function_call = response.choices[0]['message']['function_call']
  function_call_name = function_call.name
  function_call_arguments = function_call.arguments
  print(function_call_arguments)
else:
  if llm_model == "claude-2":
    print(response.choices[0]['message']['content'])
  else:
    print('No function found')

print('\n')
print(response.choices[0]['message']['content'])
print('\n')
print(json.dumps(response.json(), indent=2))

In [ ]:
print(response.choices[0]['message']['content'])

# Text splitting

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

content = "This is a piece of text. With some text and some more text."

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 10,
    chunk_overlap  = 0,
    length_function = len,
)

chunks = text_splitter.split_text(content)

print('Number of text chunks: ', len(chunks))

for chunk in chunks:
    print(chunk)

Number of text chunks:  7
This is a
piece of
text.
With some
text and
some more
text.


# Token counting

- https://platform.openai.com/tokenizer
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb

In [10]:
import tiktoken

content = "This is a piece of text. With some text and some more text."

enc = tiktoken.get_encoding("cl100k_base")
token_count = len(enc.encode(content))

print('Number of tokens: ', token_count)

Number of tokens:  15
